In [1]:
import numpy as np
import sys

In [3]:
class PartyNN(object):

    def __init__(self, learning_rate=0.1):
        self.weights_0_1 = np.random.normal(0.0, 2 ** -0.5, (2, 3))
        self.weights_1_2 = np.random.normal(0.0, 1, (1, 2))
        self.sigmoid_mapper = np.vectorize(self.sigmoid)
        self.learning_rate = np.array([learning_rate])

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs): # len=3
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)

        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        return outputs_2

    def train(self, inputs, expected_predict):
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)

        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        actual_predict = outputs_2[0]

        error_layer_2 = np.array([actual_predict - expected_predict])
        gradient_layer_2 = actual_predict * (1 - actual_predict)
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2
        self.weights_1_2 -= (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1)))) * self.learning_rate

        error_layer_1 = weights_delta_layer_2 * self.weights_1_2
        gradient_layer_1 = outputs_1 * (1 - outputs_1)
        weights_delta_layer_1 = error_layer_1 * gradient_layer_1
        self.weights_0_1 -= np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T * self.learning_rate

In [4]:
def mean_squared_error(y, Y):
    return np.mean((y - Y) ** 2)

In [5]:
train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 1),
    ([1, 0, 1], 1),
    ([1, 1, 0], 0),
    ([1, 1, 1], 1),
]

In [9]:
# to GPU, Parallel

epochs = 5000
learning_rate = 0.05

network = PartyNN(learning_rate=learning_rate)

for e in range(epochs):
    inputs_ = []
    correct_predictions = []
    for input_stat, correct_predict in train:
        network.train(np.array(input_stat), correct_predict)
        inputs_.append(np.array(input_stat))
        correct_predictions.append(np.array(correct_predict))
    
    train_loss = mean_squared_error(network.predict(np.array(inputs_).T), np.array(correct_predictions))
    sys.stdout.write("\rProgress: {}, Training loss: {}".format(str(100 * e / float(epochs))[:4], str(train_loss)[:5]))

Progress: 99.9, Training loss: 0.004

In [16]:
for input_stat, correct_predict in train:
    predict = network.predict(np.array(input_stat))
    print("For input: {} the prediction is: {}:{}, expected: {}".format(
        str(input_stat),
        str(predict),
        str(predict > .5),
        str(correct_predict == 1)))

For input: [0, 0, 0] the prediction is: [0.12968761]:[False], expected: False
For input: [0, 0, 1] the prediction is: [0.94530342]:[ True], expected: True
For input: [0, 1, 0] the prediction is: [0.00098543]:[False], expected: False
For input: [0, 1, 1] the prediction is: [0.04263897]:[False], expected: False
For input: [1, 0, 0] the prediction is: [0.9450802]:[ True], expected: True
For input: [1, 0, 1] the prediction is: [0.97274353]:[ True], expected: True
For input: [1, 1, 0] the prediction is: [0.04262131]:[False], expected: False
For input: [1, 1, 1] the prediction is: [0.92116731]:[ True], expected: True


In [17]:
network.weights_0_1

array([[ 2.34259801, -2.74154888,  2.41982307],
       [-2.68183845,  3.14624019, -2.64335414]])

In [18]:
network.weights_1_2

array([[ 3.64176713, -7.44921448]])

[Resource](https://www.youtube.com/watch?v=HA-F6cZPvrg)